##  물류 유통량 예측 경진 대회 
대회 기간 : 2021.12.06 ~ 2021.12.20
`public score : 6.3314`, `private score : 10.00293`
사용 모델 : Tabnet
대회 task : 정형데이터 - 회귀

In [1]:
!pip install pytorch-tabnet == 3.1.1

## 라이브러리 불러오기

In [2]:
import numpy as np
import pandas as pd
import torch

from torch import nn
from pytorch_tabnet.tab_model import TabNetRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold

## 데이터 불러오기

In [3]:
train = pd.read_csv('train_df.csv', encoding='cp949')
test = pd.read_csv('test_df.csv', encoding='cp949')


## 원핫인코딩

In [4]:
train_one = pd.get_dummies(train)
test_one = pd.get_dummies(test)

In [6]:
train_X = train_one.drop('INVC_CONT', axis=1)
train_y = train_one['INVC_CONT']

## 모델링

In [8]:
train_X = train_X.to_numpy()
train_y = train_y.to_numpy().reshape(-1, 1)
test_X = test_one.to_numpy()

In [ ]:
kf = KFold(n_splits=5, random_state=42, shuffle=True)
preds = []
scores = []

for trn_idx, val_idx in kf.split(train_X):
    X_trn = train_X[trn_idx]
    y_trn = train_y[trn_idx]
    X_val = train_X[val_idx]
    y_val = train_y[val_idx]

    model = TabNetRegressor(verbose=1, seed=42, optimizer_fn=torch.optim.AdamW)
    model.fit(X_train=X_trn, y_train=y_trn,
              eval_set=[(X_val, y_val)],
              patience=100, max_epochs=2000,
              eval_metric=['rmse'])
    scores.append(model.best_cost)

    pred = model.predict(test_X)
    preds.append(pred)

In [24]:
print("CV 평균 점수 : %.5f" % np.mean(scores, axis=0))

CV 평균 점수 : 5.59656


## 정답파일 생성

In [12]:
sub = pd.read_csv('sample_submission.csv')
sub

,index,INVC_CONT
0,32000,0
1,32001,0
2,32002,0
3,32003,0
4,32004,0
...,...,...
4635,36635,0
4636,36636,0
4637,36637,0
4638,36638,0


In [17]:
final_pred = np.mean(preds, axis=0)

In [20]:
sub['INVC_CONT'] = final_pred

In [28]:
sub.to_csv('tabnet.csv', index=False)